# 举例1：大模型分析工具的调用

In [1]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.messages import HumanMessage
from langchain_community.tools import MoveFileTool
# 1、获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

chat_model = ChatOpenAI(model='gpt-4o-mini')

# 2、获取工具的列表
tools = [MoveFileTool()]

# 3、因为大模型invoke调用时, 需要传入函数的列表, 所以需要将工具转换为函数：convert_to_openai_function()
functions = [convert_to_openai_function(t) for t in tools]

# 4、获取消息列表
messages = [HumanMessage(content='将文件a移动到桌面')]

# 5、调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools # 不支持
    functions=functions,
)

print(response)
1

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 76, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CazSAs7HXquZYZoeVNR55lWjAsZXM', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None} id='run--75424704-35a3-412b-a8fe-5bcb688e3f6a-0' usage_metadata={'input_tokens': 76, 'output_tokens': 27, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


作为对比

In [2]:
# 4、获取消息列表
messages = [HumanMessage(content='帮我查询一下明天北京的天气')]

# 5、调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools # 不支持
    functions=functions,
)

print(response)

content='抱歉，我无法直接查询当前的天气信息，但你可以通过天气应用程序或者访问天气网站来获得最新的天气预报。建议查看常用的天气服务，如中国天气网、天气通或其他相关应用来获取明天北京的天气信息。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 78, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CazUEu6EGxjvLStOrhujdckkQ58iC', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--8e928d29-c12a-4558-987f-cd941901504d-0' usage_metadata={'input_tokens': 78, 'output_tokens': 58, 'total_tokens': 136, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


通过上面两个测试发现, 得到AIMessage的核心属性如下：
    
1、如果分析出需要调用对应的工具：

content：信息为空. 因为大模型要调用工具, 所以就不会直接返回信息给用户

additional_kwargs：包含function_call字段, 指明具体函数调用的参数和函数名. 比如：

additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None}

2、如果分析出不需要调用对应的工具：

content：信息不为空

additional_kwargs：不包含function_call字段

# 举例2：如何调用具体大模型分析出来的工具

说明：

1、大模型与Agent的核心区别：是否涉及到工具的调用

2、针对于大模型, 仅能分析出要调用的工具, 但是此工具（或函数）不能真正地执行

针对于Agent：除了分析出要调用的工具之外, 还可以执行具体的工具（或函数）
    


In [14]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.messages import HumanMessage
from langchain_community.tools import MoveFileTool
# 1、获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

chat_model = ChatOpenAI(model='gpt-4o-mini')

# 2、获取工具的列表
tools = [MoveFileTool()]

# 3、因为大模型invoke调用时, 需要传入函数的列表, 所以需要将工具转换为函数：convert_to_openai_function()
functions = [convert_to_openai_function(t) for t in tools]

# 4、获取消息列表
messages = [HumanMessage(content=r'将当前目录下的文件a.txt移动到C:\Users\leon\Desktop')]

# 5、调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools # 不支持
    functions=functions,
)

# print(response)

步骤1：分析下要调用哪个工具或函数

In [17]:
import json

if 'function_call' in response.additional_kwargs:
    # 如果模型有调用函数, 那么就获取相关的工具和参数
    tool_name = response.additional_kwargs['function_call']['name']
    # tool_args = response.additional_kwargs['function_call']['arguments']
    tool_args = json.loads(response.additional_kwargs['function_call']['arguments'])
    print(f'调用工具：{tool_name} \n参数：{tool_args}')
else:
    # 如果模型没有调用函数, 那么就获取模型输出的文本内容
    print(f'模型回复：{response.content}')

调用工具：move_file 
参数：{'source_path': 'a.txt', 'destination_path': 'C:\\Users\\leon\\Desktop\\a.txt'}


步骤2：调用对应的工具

In [18]:
if 'move_file' in response.additional_kwargs['function_call']['name']:
    tool = MoveFileTool()
    result = tool.run(tool_args)  # 调用工具
    print('工具执行的结果', result)

工具执行的结果 File moved successfully from a.txt to C:\Users\leon\Desktop\a.txt.
